In [11]:
!pip install keras

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import warnings

# Tắt các cảnh báo không cần thiết để output gọn gàng hơn
warnings.filterwarnings('ignore')

# --- PHẦN 1: TIỀN XỬ LÝ DỮ LIỆU ---

def preprocess_data(df, batch_size=1000):
    from tqdm import tqdm

    df.iloc[:, -1] = df.iloc[:, -1].apply(lambda x: 0 if x == 'normal' else 1)
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1].to_numpy()

    categorical_features = [1, 2, 3]
    preprocessor = ColumnTransformer(
        transformers=[('cat', OneHotEncoder(handle_unknown='ignore', sparse=False), categorical_features)],
        remainder='passthrough'
    )
    
    X_transformed = preprocessor.fit_transform(X)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X_transformed)

    # Chia thành batch nhỏ để resize tránh tràn bộ nhớ
    num_samples = X_scaled.shape[0]
    resized_images = []

    print("Đang resize ảnh theo từng batch nhỏ...")
    for i in tqdm(range(0, num_samples, batch_size)):
        batch = X_scaled[i:i+batch_size]
        reshaped = np.reshape(batch, (-1, 11, 11))  # batch_size x 11 x 11
        rgb = np.stack([reshaped] * 3, axis=-1)     # batch_size x 11 x 11 x 3
        resized = tf.image.resize(rgb, (224, 224))  # batch_size x 224 x 224 x 3
        resized_images.append(resized.numpy())

    X_resized = np.concatenate(resized_images, axis=0)
    print(f"Hoàn tất tiền xử lý, hình dạng dữ liệu X: {X_resized.shape}, y: {y.shape}")
    return X_resized, y, preprocessor


# --- PHẦN 2: XÂY DỰNG MÔ HÌNH ---

def build_vgg16_model(input_shape=(224, 224, 3)):
    """
    Hàm này xây dựng mô hình học chuyển giao dựa trên VGG-16:
    1. Tải mô hình VGG-16 đã huấn luyện trước trên ImageNet, bỏ lớp cuối (top).
    2. Đóng băng tất cả các lớp của VGG-16 để không huấn luyện lại chúng.
    3. Thêm các lớp Fully Connected (Dense) mới để phù hợp bài toán.
    """
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    for layer in base_model.layers:
        layer.trainable = False  # Đóng băng toàn bộ các lớp VGG-16

    x = base_model.output
    x = Flatten()(x)  # Chuyển đầu ra từ feature maps thành vector

    # DNN 2 tầng ẩn có regularization
    x = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(x)
    x = Dense(8, activation='relu', kernel_regularizer=l2(0.001))(x)

    # Output layer: 1 node, sigmoid cho bài toán phân loại nhị phân
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    print("Mô hình đã được xây dựng đúng theo bài báo:")
    model.summary()
    return model

# --- PHẦN 3: HUẤN LUYỆN VÀ ĐÁNH GIÁ ---

# Đặt tên cột cho bộ dữ liệu NSL-KDD
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

# Tải dữ liệu huấn luyện và kiểm thử
print("Đang tải dữ liệu...")
# Lưu ý: Bài báo sử dụng 10% dữ liệu. Để đơn giản, chúng ta sẽ dùng toàn bộ.
# Nếu bạn muốn kết quả sát nhất, hãy lấy 10% mẫu ngẫu nhiên từ các tệp này.
train_df = pd.read_csv("/kaggle/input/nsl-kdd/TanCongMangVGG16/dataset/KDDTrain+.txt", header=None, names=col_names)
test_df = pd.read_csv("/kaggle/input/nsl-kdd/TanCongMangVGG16/dataset/KDDTest+.txt", header=None, names=col_names)
# test21_df = pd.read_csv('/kaggle/input/nsl-kdd/TanCongMangVGG16/dataset/KDDTest-21.txt', header=None, names=col_names)
print("Tải dữ liệu thành công.")
target_names= ["normal","attack"]
# Tiền xử lý dữ liệu
print("\nBắt đầu tiền xử lý dữ liệu huấn luyện...")
X_train, y_train, preprocessor = preprocess_data(train_df)
X_test, y_test, _ = preprocess_data(test_df)

print("\nBắt đầu tiền xử lý dữ liệu kiểm thử...")
X_test, y_test = preprocess_data(test_df)

# Xây dựng mô hình
model = build_vgg16_model()

# Biên dịch (compile) mô hình
# Sử dụng các tham số như trong bài báo 
optimizer = Adam(learning_rate=0.001, decay=1e-5)
model.compile(optimizer=optimizer, 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Cấu hình Early Stopping để tránh overfitting 
early_stopping = EarlyStopping(monitor='val_loss', 
                               min_delta=1e-4, 
                               patience=10, 
                               restore_best_weights=True)

# Huấn luyện mô hình
print("\nBắt đầu quá trình huấn luyện...")
history = model.fit(X_train, y_train,
                    epochs=50, # Đặt số epochs lớn, EarlyStopping sẽ dừng khi cần
                    batch_size=64, # 
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

# Đánh giá mô hình trên tập kiểm thử
print("\nBắt đầu đánh giá mô hình...")
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype("int32")

# print("\n--- KẾT QUẢ ĐÁNH GIÁ TRÊN TẬP KDDTest+ ---")
print(classification_report(y_test, y_pred, target_names=target_names))

# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f_score = f1_score(y_test, y_pred)

# print(f"Accuracy:  {accuracy*100:.2f}%")
# print(f"Precision: {precision*100:.2f}%")
# print(f"Recall:    {recall*100:.2f}%")
# print(f"F1-Score:  {f_score*100:.2f}%")

Đang tải dữ liệu...
Tải dữ liệu thành công.

Bắt đầu tiền xử lý dữ liệu huấn luyện...
